prostate cancer substructure screeening


In [ ]:
!pip install -q condacolab

import condacolab
condacolab.install()

!mamba install -q -c conda-forge rdkit

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:43
🔁 Restarting kernel...
  Package               Version  Build                Channel                    Size
───────────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────────

  boost                  1.74.0  py37h6dcda5c_3       conda-forge/linux-64     342 KB
  boost-cpp              1.74.0  hc6e9bd1_3           conda-forge/linux-64      16 MB
  cairo                  1.16.0  h6cf1ce9_1008        conda-forge/linux-64       1 MB
  cycler                 0.10.0  py_2                 conda-forge/noarch         9 KB
  fontconfig             2.13.1  hba837de_1005        conda-forge/linux-64     357 KB
  freetype               2.10.4  h0708190_1           conda-forge/linux

In [ ]:
# Import required modules
import numpy as np
import pandas as pd
import os
import rdkit as rd
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import RDConfig
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.rdMolDescriptors import GetAtomPairFingerprint
from rdkit.Chem.AtomPairs import Torsions
from collections import Counter

In [ ]:

import sklearn as sk
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
uploaded = files.upload()

Saving Prostate_cancer_substr_screening.csv to Prostate_cancer_substr_screening.csv


In [ ]:
prostate_small_molecules= pd.read_csv("Prostate_cancer_substr_screening.csv")


In [ ]:
prostate_small_molecules.head(2)

,smiles,activity
0,CO[C@H]1C[C@@H]2CC[C@@H](C)[C@@](O)(O2)C(=O)C(...,0
1,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,0


In [ ]:
#Convert smiles to mol 
prostate_small_molecules["mol"] = [Chem.MolFromSmiles(x) for x in prostate_small_molecules["smiles"]]



MORGAN FINGERPRINT

In [ ]:
prostate_small_molecules["morgan_fingerprint"] = [Chem.GetMorganFingerprintAsBitVect(m,2,nBits=2048) for m in prostate_small_molecules['mol']]
morgan_fingerprint = [Chem.GetMorganFingerprintAsBitVect(m,2,nBits=2048) for m in prostate_small_molecules['mol']]
morgan_fingerprint_np = []
for fp in morgan_fingerprint:
  arr=np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp,arr)
  morgan_fingerprint_np.append(arr)

# DAYLIGHT FINGERPRINT

In [ ]:
prostate_small_molecules["rdkit_fingerprint"] = [Chem.RDKFingerprint(m) for m in prostate_small_molecules["mol"]]
rdkit_fingerprint = [Chem.RDKFingerprint(m) for m in prostate_small_molecules["mol"]]
rdkit_fingerprint_np = []
for fp in rdkit_fingerprint:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  rdkit_fingerprint_np.append(arr)

ATOM-PAIR FINGERPRINT

In [ ]:
prostate_small_molecules["AtomPair_Fingerprint"] = [Chem.GetHashedAtomPairFingerprintAsBitVect(m) for m in prostate_small_molecules["mol"]]
AtomPair_Fingerprint = [Chem.GetHashedAtomPairFingerprintAsBitVect(m) for m in prostate_small_molecules["mol"]]
AtomPair_Fingerprint_np = []
for fp in AtomPair_Fingerprint:
  arr = np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  AtomPair_Fingerprint_np.append(arr)

TORSION FINGERPRINT

In [ ]:
prostate_small_molecules["TorsionFingerprint"] = [Chem.GetHashedTopologicalTorsionFingerprintAsBitVect(m) for m in prostate_small_molecules['mol']]
TorsionFingerprint = [Chem.GetHashedTopologicalTorsionFingerprintAsBitVect(m) for m in prostate_small_molecules['mol']]
TorsionFingerprint_np = []
for fp in TorsionFingerprint:
  arr=np.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp,arr)
  TorsionFingerprint_np.append(arr)

In [ ]:
x_morgan = morgan_fingerprint_np
x_rdkit = rdkit_fingerprint_np
x_torsion = TorsionFingerprint_np
x_Atompair = AtomPair_Fingerprint_np
y = prostate_small_molecules.activity.values

In [ ]:
X_morgan_train, X_morgan_test, y_morgan_train, y_morgan_test = train_test_split(x_morgan,y,test_size=0.33, random_state=42)
X_rdkit_train, X_rdkit_test, y_rdkit_train, y_rdkit_test = train_test_split(x_rdkit,y,test_size=0.33, random_state=42)
X_torsion_train, X_torsion_test, y_torsion_train, y_torsion_test = train_test_split(x_torsion, y, test_size=0.33, random_state = 42)
X_Atompair_train, X_Atompair_test, y_Atompair_train, y_Atompair_test = train_test_split(x_Atompair, y, test_size=0.33, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)


In [ ]:
# Default parameters of current Random Forest
from pprint import pprint
pprint(rf.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

RANDOM FOREST

In [ ]:
# No. of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]

# No.of features to be considered at every split
max_features = ['auto', 'log2']

# Maximum number of levels in tress
max_depth = [int(x) for x in np.linspace(start=10, stop=110, num=11)]
max_depth.append(None)

# Mininum no. of samples required to split the nodes
min_sample_split = [2,5,10]

# Minimum number of samples required at each leaf node
min_sample_leaf = [1,2,4]

# Method to select samples for training each tree
boostrap = [True, False]

In [ ]:
# Creat the random_grid

random_grid ={'n_estimators':n_estimators,
              'max_depth':max_depth,
              'max_features':max_features,
              'min_samples_split':min_sample_split,
              'min_samples_leaf':min_sample_leaf
                            }

In [ ]:
random_grid

{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'log2'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [ ]:
# Utilize random_grid to identify the best hyper parameters, 
#set cross validation as 3 for using 3 fold cross validation, 
#and iter as 100 to search across 100 different combinations
# use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter=50, cv=3, verbose=2, random_state=42, n_jobs=-1)


In [ ]:
rf_random.fit(X_morgan_train, y_morgan_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.7min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
rf_random.best_params_

{'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [ ]:
forestOpt = RandomForestClassifier(random_state = 42, max_depth = 20,     n_estimators = 1000, min_samples_split = 1.0, min_samples_leaf = 2)

In [ ]:
modelOpt = forestOpt.fit(X_morgan_train, y_morgan_train)
y_pred = modelOpt.predict(X_morgan_test)

In [ ]:
# Accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_morgan_test, y_pred)

0.7676056338028169

Compare base model with best random search model

In [ ]:
base_model = RandomForestClassifier(n_estimators=100, random_state=42, min_samples_split = 2, min_samples_leaf = 1)
bm = base_model.fit(X_morgan_train, y_morgan_train)
y_pred_bm = bm.predict(X_morgan_test)
accuracy_score(y_morgan_test, y_pred)

0.7676056338028169

GRID SEARCH

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap':[True],
    'n_estimators':[100,200,300,1000],
    'max_depth':[80,90,100,110],
    'min_samples_leaf':[3,4,5],
    'min_samples_split':[8,10,12],
    }
rf = RandomForestClassifier()

# Instantiate grid model

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, verbose=2)

In [ ]:
# Fit grid search into data

grid_search.fit(X_morgan_train, y_morgan_train)
grid_search.best_params_






Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.2s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s



[CV] bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.2s
[CV] bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.2s
[CV] bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=200, total=   0.3s
[CV] bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=200, total=   0.4s
[CV] bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=True, max_depth=80, min_samples_leaf=3, min_samples_split=8, n_estimators=200

[Parallel(n_jobs=1)]: Done 432 out of 432 | elapsed:  4.8min finished


{'bootstrap': True,
 'max_depth': 80,
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 100}

In [ ]:
#best_grid = grid_search.best_estimator_
#grid_accuracy = evaluate(best_grid, X_morgan_test, y_morgan_test)

Model Performance
Average Error: 0.2183 degrees.
Accuracy = nan%.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [ ]:
# View confusion matrix for test data and predictions
#confusion_matrix(y_test, y_pred_test)

In [ ]:
#from sklearn.metrics import confusion_matrix


In [ ]:
#base_model = RandomForestClassifier(n_estimators=100, random_state=42, min_samples_split = 2, min_samples_leaf = 1)
#bm = base_model.fit(X_morgan_train, y_morgan_train)
#y_pred_bm = bm.predict(X_morgan_test)
#accuracy_score(y_morgan_test, y_pred)